## Passing workflows between functions

Demo to use workflows in functions for end to end image analysis

In [4]:
import os
import pyclesperanto_prototype as cle
from napari_workflows import Workflow

cle.get_device()


<NVIDIA GeForce RTX 3080 on Platform: NVIDIA CUDA (1 refs)>

In [5]:
#download sample data from https://zenodo.org/records/14903188
img_path = "./sample_data/RBC_tiny.czi"

#make dir if not exists
if not os.path.exists("./sample_data"):
    os.makedirs("./sample_data")

import urllib.request

url = "https://zenodo.org/records/14903188/files/RBC_tiny.czi?download=1"
if not os.path.exists(img_path):
    print("Downloading file")
    urllib.request.urlretrieve(url, img_path)
    print(f"Downloaded at : {img_path}")
else:
    print(f"File already exists at : {img_path}")

File already exists at : ./sample_data/RBC_tiny.czi


In [6]:
#absolute path for saving
save_path = "C:/Users/Pradeep/nap/napari_lattice/notebooks/sample_data"
#make dir if not exists
if not os.path.exists(save_path):
    os.makedirs(save_path)


To design end to end image analysis workflows, we can use napari-workflows 

https://github.com/haesleinhuepf/napari-workflows

Lets apply a gaussian blur filter using pyclesperanto library

In [16]:
#import gaussian filter from pyclesperanto
#We initialise a workflow
gaussian_workflow = Workflow()

#For napari-lattice, the input image should always be of the name "deskewed_image"

input_arg = "deskewed_image"
#define a simple gaussian filter
task_name = "gaussian"
gaussian_blur_sigma_x = 5
gaussian_blur_sigma_y = 5
gaussian_blur_sigma_z = 2

gaussian_workflow.set(task_name, cle.gaussian_blur, 
                      input_arg, sigma_x = gaussian_blur_sigma_x,
                      sigma_y = gaussian_blur_sigma_y,
                      sigma_z = gaussian_blur_sigma_z)

#Printing a workflow will show you the arguments
print(gaussian_workflow)

Workflow:
gaussian <- (<function gaussian_blur at 0x000002367F907BE0>, 'deskewed_image', None, 5, 5, 2)



In [17]:
from lls_core import LatticeData

params = LatticeData(
  input_image=img_path,
  save_dir=save_path,
  workflow=gaussian_workflow,
)

print(params)

[INFO:2025-08-27 16:43:03,536] Processing File ./sample_data/RBC_tiny.czi


input_image=<xarray.DataArray 'transpose-04519f8a37cc3af50256674a3d06fa88' (T: 1, C: 1,
                                                                Z: 834, Y: 118,
                                                                X: 209)>
dask.array<transpose, shape=(1, 1, 834, 118, 209), dtype=uint16, chunksize=(1, 1, 834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
  * C        (C) <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Dimensions without coordinates: T
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x0000023640A7FBA0> skew=<DeskewDirection.Y: 2> angle=30.0 physical_pixel_sizes=DefinedPixelSizes(X=0.14499219272808386, Y=0.14499219272808386, Z=0.3) derived=DerivedDeskewFields(deskew_vol_shape=(59, 1828, 209), deskew_affine_transform=<pyclesperanto_

In [15]:
# when running isave, if params.workflow has a workflow, then it will run the analysis workflow
params.workflow

In [9]:
params.save()

Timepoints:   0%|          | 0/1 [00:00<?, ?it/s]

INFO: blockdim levels (1) < subsamp levels (3): First-level block size (4, 256, 256) will be used for all levels


Timepoints: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


This should save a deskewed image with gaussian blur applied

Lets apply a workflow, where we perform 
- median filter
- binarise using otsu algorithm with threshold of 1000
- label each object using connected components labeling-
- measure each object using connected components

As we have multiple steps, we can define it in a function and then call this function in a workflow. 

We would like to save the deskewed image, segmented image and the measurements as a csv file, so we return all 3.

In [7]:
from lls_core import LatticeData

image_seg_workflow = Workflow()

#Otsu thresholding
input_arg = "deskewed_image"
#define a simple gaussian filter
task_name = "measurement"

from skimage.measure import regionprops_table
import numpy as np
import pandas as pd 

def image_seg_measure(input_image):
    # Apply median filter
    median_filtered = cle.median_sphere(input_image, radius_x=2, radius_y=2, radius_z=2)

    # Apply Otsu thresholding
    binary_image = cle.threshold_otsu(median_filtered)

    # Apply connected components labeling
    labeled_image = cle.connected_components_labeling_box(binary_image)
    
    #convert labeled_image to np array
    labeled_image_np = np.array(labeled_image)
    
    #measure using regionprops_table
    props = regionprops_table(labeled_image_np, input_image,
                              properties=['label', 'bbox','area', 
                                          'intensity_mean','intensity_std'])
    #convert to pandas dataframe
    import pandas as pd
    props_df = pd.DataFrame(props)
    return (input_image, labeled_image_np, props_df)

#Apply function
image_seg_workflow.set(task_name, image_seg_measure,  input_arg )

print(image_seg_workflow)

c:\Users\Pradeep\.conda\envs\napari-lattice\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Workflow:
measurement <- (<function image_seg_measure at 0x000001846AD8B2E0>, 'deskewed_image')



In [8]:
#Before we run this, lets look at whats in the directory
os.listdir(save_path)

['RBC_tiny.czi']

In [9]:
params_workflow = LatticeData(
  input_image=img_path,
  save_dir=save_path,
  workflow=image_seg_workflow,
)

params_workflow.save()

[INFO:2025-08-27 17:49:46,967] Processing File ./sample_data/RBC_tiny.czi
Timepoints:   0%|          | 0/1 [00:00<?, ?it/s][INFO:2025-08-27 17:49:47,293] Processing File <xarray.DataArray 'transpose-fb98b245a6efefda1c33584fac697038' (Z: 834, Y: 118,
                                                                X: 209)>
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000001842549D0D0>


INFO: blockdim levels (1) < subsamp levels (3): First-level block size (4, 256, 256) will be used for all levels
INFO: blockdim levels (1) < subsamp levels (3): First-level block size (4, 256, 256) will be used for all levels


Timepoints: 100%|██████████| 1/1 [00:04<00:00,  4.03s/it]


In [10]:
#list files in save_path
os.listdir(save_path)


['RBC_tiny.czi',
 'RBC_tiny_deskewed.h5',
 'RBC_tiny_deskewed.xml',
 'RBC_tiny_deskewed_1.h5',
 'RBC_tiny_deskewed_1.xml',
 'RBC_tiny_deskewed_output_2.csv']

- RBC_tiny_deskewed.h5 is the deskewed image
- RBC_tiny_deskewed_1.h5 is the label image
- RBC_tiny_deskewed_output_2.csv is the table with measurements


Test code to inspect the outputs of workflow

In [145]:
for slice in params_workflow.process_workflow():
    first_slice = slice
    break

In [146]:
slice

('slices',
 <generator object LatticeData.process_workflow.<locals>._generator at 0x000002363A268350>)

In [147]:
data_set = next(first_slice[1])
data_set

Timepoints:   0%|          | 0/1 [00:00<?, ?it/s][INFO:2025-08-27 17:45:42,814] Processing File <xarray.DataArray 'transpose-81c08323b456b9acae2e371254043f37' (Z: 834, Y: 118,
                                                                X: 209)>
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000002363A2D3330>


ProcessedSlice(data=(array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.

In [149]:
data_set.data[2].apply(Series.explode)

AttributeError: 'dict' object has no attribute 'apply'

In [ ]:
#data_set.data[2]
pd.DataFrame(data_set.data[2]).apply(Series.explode)